# ZPEM2509 Astrophysics Lab #2: Eclipsing Binaries 

Developed by Dr. Simon Murphy, UNSW Canberra ([s.murphy@adfa.edu.au](mailto:s.murphy@adfa.edu.au))

Please read the background material in the lab script and complete the online simulations in Part I prior to starting this exercise.

<br>

# Part II: Fundamental properties of the young eclipsing binary THOR 42

<br>

Now that we have an understanding of how the physical and orbital parameters of an eclipsing binary affect its light curve, we can combine the light curve with radial velocity measurements to derive the fundamental properties (mass, radius, age, temperature, luminosity) of both stars. 

Our target of interest is the low-mass eclipsing binary __THOR 42__ (otherwise known as CRTS J055255.7-004426) which is a member of a young group of stars called the [32 Orionis Moving Group](https://ui.adsabs.harvard.edu/abs/2017MNRAS.468.1198B/abstract). THOR 42 comprises two M dwarf stars which are smaller, less massive, fainter and cooler than the Sun. You can find more information on the system in the [Simbad database](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=THOR+42&submit=SIMBAD+search). 

<div style="text-align:center;margin:30px">
    <table><tr>
        <td><img src="orion.png" style="height:350px"></td>
        <td><img src="THOR42.jpg" style="height:350px"></td>
    </tr></table>
    Location of THOR 42 in the constellation of Orion.
</div>

THOR 42 was observed by the NASA *TESS* ([*Transiting Exoplanet Survey Satellite*](https://www.nasa.gov/tess-transiting-exoplanet-survey-satellite)) mission between 2018 December 15 and 2019 January 6 as part of its all-sky survey. Although *TESS* lacks the spatial resolution of larger ground-based telescopes, it has the advantage of incredibly stable image quality and the ability to record images every 30 min (2 min for some high priority stars), 24 hours a day while observing a particular field. 

In this lab we will use *TESS* data to construct a light curve for THOR 42, then analyse radial velocities from the [ANU 2.3-m telescope](https://rsaa.anu.edu.au/observatories/telescopes/anu-23m-telescope) at [Siding Spring Observatory](https://goo.gl/maps/UBrdy1XS57sKuqrR9) near Coonabarabran, NSW. First, let's load some useful Python  modules to read the data and make plots. Remember to press `SHIFT` + `RETURN` to run the code and move to the next cell.

<br>


In [ ]:
# Import various libraries
%matplotlib widget
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from interactive_figures import RVCurve, LightCurve, MassRadiusDiagram

## A. Light curve analysis

<br>We begin by loading the *TESS* photometry from a text file. This file has 6 columns - the Barycentric Julian Date (BJD), the flux from the *TESS* pixels containing THOR 42, a flux error, the flux converted to a magnitude, a magnitude error and a flag (0 or 1) indicating whether the measurements are good or bad. Here we are only interested in the BJD, flux and flag columns. 

Note: rather than standard geocentric Julian Dates (JD), we use Barycentric Julian Dates (BJD) taken at the centre of mass of the solar system (approximately the centre of the Sun). This is necessary otherwise the eclipse times recorded by *TESS* would be up to $\pm$8.3 min different depending on which side of the Sun the Earth was at the time of observation.

In [ ]:
# Load the TESS photometry
tess = Table.read('TESS.txt', format='ascii')
# Only keep epochs with no issues (FLAG = 0)
good = tess['FLAG'] == 0 # this is an array of True or False values
bjd = tess['BJD'][good]
flux = tess['FLUX'][good]

And now we plot the time-series light curve. You can interact with the figure using the toolbar on the left; the zoom to rectangle (second button from bottom) and pan (4-way arrow) are particularly useful. $x$ and $y$ coordinates are displayed at the bottom of the plot.

In [ ]:
# Plot the TESS time series
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
plt.plot(bjd, flux, 'o-', ms=2, lw=0.5, color='black', label='Good')
# Also plot the rejected epochs in red. 
# The '~' operator does the inverse and selects the not good (i.e. bad) points
plt.plot(tess['BJD'][~good], tess['FLUX'][~good], 'o', ms=2, color='tab:red', label='Bad')
plt.xlabel('BJD $-$ 2450000')
plt.ylabel('Flux (normalised)')
plt.title('$TESS$ light curve')
plt.legend()
plt.show()

The deep primary and shallower secondary eclipses are immediately visible in the light curve, as is substantial out-of-eclipse brightness variation. 

The red data points have been rejected for failing quality checks (`FLAG=1`). The large group of bad points between $8475 < \mathrm{BJD} - 2450000 < 8479$ are contaminated by Earth-shine reflecting off the spacecraft's sun shade and the bad points every 3 days are due to thruster firings to dump angular momentum from the reaction wheels (similar to gyroscopes) used to keep the spacecraft pointing accurately.

__Zoom in and explore the plot, saving a copy for your report. Estimate an approximate period ($P$, in days) and a time of primary eclipse ($t_{0}$, BJD).__ 

We can use these values to create a phased (sometimes called folded) light curve. The orbital phase $\phi$ is defined as:

\begin{align}
\phi = (\mathrm{BJD} - t_{0})/P - \mathrm{floor}([\mathrm{BJD} - t_{0}]/P)
\end{align}

where the $\mathrm{floor()}$ function returns the rounded down integer value. The phase therefore varies between 0 and 1 as the stars move in their orbits and our choice of $t_{0}$ places the primary eclipse at $\phi = 0$ (also  $\phi = 1$). 

__Use the figure below to create a phase-folded light curve for THOR 42 with the $P$ and $t_{0}$ values you estimated from the raw light curve.__ 

<br>


In [ ]:
# Plot the interactive figure
LightCurve(bjd, flux)

Given the short (~2 hr) duration of the eclipses and the 30 min cadence of the *TESS* images, it is likely your initial choices of $P$ and $t_{0}$ are not optimal. We can assess how well the light curve is phased by computing the ['string length'](https://academic.oup.com/mnras/article/203/4/917/1029604) of the light curve. To do this we simply connect up all the points in order of phase and compute the total length. The best period should have the shortest string length. 

__Turn on the string length checkbox in the figure and adjust the period to minimise the length. Once you have a good period estimate you can adjust $t_{0}$ so that the primary eclipse occurs at $\phi=0$. You may wish to zoom in and turn on the grid to help with this. Record these values.__

__Assuming the two stars have different radii, what does the shape of the primary eclipse tell us about the inclination $i$ of the system to the line of sight? What do the different eclipse depths tell us about the temperatures of the stars? What does the phase of the secondary eclipse tell us about the eccentricity $e$ of the orbits? You may wish to refer back to the online light curve simulator for guidance.__ 

Comparing the *TESS* light curve to those from the simulator you will notice THOR 42 shows a considerable drop in brightness either side of primary eclipse. This is thought to be due to a large (10\% of the stellar surface) and cool (therefore darker by the Stefan-Boltzmann law) star spot on the surface of the primary which rotates into view just before the eclipse. Since THOR 42 is such a compact binary system we expect the rotation rate of each star to be the same as the orbital period (this is called *tidal synchronisation*) so the spot will never fall out of phase with the eclipse.



## B. Eclipse observations

<br>If you were to use the $P$ and $t_0$ values you just determined to phase another light curve you would find that  the eclipses no longer lined up exactly with $\phi=0$ and $\phi=0.5$. This is because the *TESS* data are sparsely sampled and cover a short baseline (22 days) at a recent epoch, so any small errors in $P$ or $t_0$ will accumulate when these are applied to much earlier data like the radial velocities.

Rather than finding a combination of $P$ and $t_0$ which suits all the data, we need to break the degeneracy between $P$ and $t_0$ and use a well-sampled light curve of the primary eclipse to accurately determine $t_0$ without knowing the period. Luckily, we have 50-sec cadence observations from the ANU [SkyMapper telescope](http://skymapper.anu.edu.au/about-skymapper/) at Siding Spring Observatory, which captured a single primary eclipse on 2017 December 13:



In [ ]:
# Load the SkyMapper photometry
sm = Table.read('SkyMapper.txt', format='ascii')
# Only consider the fraction of a day around the eclipse
sm['BJD'] -= 8101
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
plt.plot(sm['BJD'], sm['dmag'], 'o', ms=2, color='black')
plt.xlabel('BJD $-$ 2458101.0')
plt.ylabel('Differential $i$ magnitude')
plt.title('SkyMapper primary eclipse (2017 Dec 13)')
plt.xlim(sm['BJD'][0],sm['BJD'][-1])
plt.ylim(plt.ylim()[::-1]) # Reverse the y-axis
t0_line = plt.axvline(x=999, color='tab:green')
plt.show()

Here the brightness of the system is measured in *magnitudes*, not flux (a relative flux of 1.0 would correspond to a magnitude of zero in this plot). Recall that a larger magnitude indicates a *fainter* measurement. 

__Use the figure to accurately determine the point of mid-eclipse, $t_0$. Include a copy of the figure in your report. You may wish to add a vertical line at your chosen $t_0$ value; this can be done by running the following command:__

In [ ]:
t0_line.set_xdata(0.0) # Replace 0.0 with your chosen t0 (BJD - 2458101)

__Use your new $t_{0}$ value to update the phased light curve, adjusting the period to align the eclipses with $\phi=0.0$ and $\phi=0.5$. When you are happy with the results, record the values of $P$ and $t_0$ and save a copy of the phased light curve for your report.__



## C. Radial velocity curves

<br>Now that we have a good idea of the period and eclipse timing we can analyse the radial velocity data. We again read it in from a text file containing the BJD time of observation, the radial velocities for each star, their uncertainties and a flag. We reject those measurements that were taken too close to the eclipses and have erroneous secondary velocities. This leaves 65 primary and 52 secondary velocities.


In [ ]:
# Load the RV data 
rv = Table.read('RV.txt', format='ascii')
t1 = rv['BJD']
rv1 = rv['RV1']
# Inflate the velocity errors to force the reduced chi-squared
# to be approximately 1 for the best fitting model.
# This is not best practice, see https://arxiv.org/pdf/1012.3754.pdf
e_rv1 = np.sqrt(rv['E_RV1']**2 + 3.5**2)
# Use only those secondary epochs with good velocities (FLAG = 0)
good = rv['FLAG'] == 0 
t2 = rv['BJD'][good]
rv2 = rv['RV2'][good]
# Again, inflate the errors
e_rv2 = np.sqrt(rv['E_RV2'][good]**2 + 5.0**2)

First, we plot the radial velocities of both stars versus time:


In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
plt.plot(t1, rv1, 'o', ms=4, color='tab:red', label='Primary')
plt.plot(t2, rv2, 'o', ms=4, color='tab:blue', label='Secondary')
plt.xlabel('BJD $-$ 2450000')
plt.ylabel('Radial velocity (km s$^{-1}$)')
plt.title('WiFeS radial velocities')
plt.legend()
fig.show()

The first measurement was made at $\mathrm{BJD}\approx2457319$ (2015 October 23) and the velocities had swapped signs at the time the second measurement was made almost a year later. This immediately identified THOR 42 as a spectroscopic binary and it was observed frequently over the following 18 months, often multiple times per night. 

Both stars will obey Kepler's Laws as they orbit each other. For stars moving in elliptical orbits with period $P$, eccentricity $e$, inclination $i$ and semimajor axes $a_1$ and $a_2$, it can be shown that the *observed* radial velocities follow the following relations:

\begin{align}
v_{r}(t) &= v_{\mathrm{sys}} + K[e\cos\omega + \cos(\omega + \phi)] \\\\
\mathrm{with}\ K_1 &= \frac{2\pi a_1 \sin i}{P \sqrt{1-e^{2}}} \ \ \mathrm{and}\ \ K_2 = \frac{2\pi a_2 \sin i}{P \sqrt{1-e^{2}}}\\
\end{align}

and where $v_{\mathrm{sys}}$ is the constant *systemic* or centre-of-mass velocity and $\omega$ relates to the angle between peri-astron (closest approach between the stars) and the line of sight. Both $\omega$ and $i$ serve to orient the orbit relative to the plane of the sky. For a circular orbit ($e=0$) the expression simplifies to a simple (co)sine curve and $\omega$ is undefined. $K_1$ and $K_2$ are called the radial velocity semi-amplitudes. We cannot be sure of the *true* semi-amplitudes because the orbital inclination $\sin i$ is still unknown, but for an eclipsing system we know it must be close to $i=90^{\circ}$.

Although we could fit a model for the radial velocities to the time domain data, it makes more sense to phase the velocities to match the light curve. 

__Use the figure below to create velocity curves for THOR 42 with the improved $P$ and $t_0$ values you estimated above. Use the sliders to fit radial velocity models $v_{r}(t)$ to the observed velocities, aiming to minimise the $\chi^2$ statistic, defined as:__

\begin{align}
\chi^{2} = \sum_{i}{\frac{(O_{i} - M_{i}(\theta))^{2}}{\sigma_{i}^{2}}}
\end{align}

where we sum over the individual observations $O$, model values $M$ (evaluated at parameters $\theta$) and uncertainties $\sigma$. The reduced $\chi^2$ statistic displayed on the plot is simply $\chi_{r}^2 = \chi^{2}/ (N-P)$ for $N$ data points and $P$ model parameters. A $\chi_{r}^2$ value close to 1.0 indicates a good fit to the observations.

<br>

In [ ]:
# Plot the interactive figure
RVCurve(t1, rv1, e_rv1, t2, rv2, e_rv2)

__When you are happy with the results, record the values of $K_1$, $K_2$, $v_{\mathrm{sys}}$, $e$ and $\omega$,
  and save a copy of the velocity curves for your report. It should be possible to obtain $\chi_{r}^2$ values around 1.0 for both the primary and secondary velocities.__  
  
__Once this is complete, verify that your initial values of $P$ and $t_0$ from just the *TESS* observations give a much poorer fit to the velocities.__

We now have everything we need to determine the absolute masses and radii of both stars.
  

## D. Masses and separation of THOR 42

<br>From centre-of-mass arguments, it is easy to show that for two stars of mass $M_1$ and $M_2$;

\begin{align}
a_1 M_1 = a_2 M_2 \ \ \mathrm{and}\ \ M_1 K_1 = M_2 K_2
\end{align}

for semi-major axes $(a_1, a_2)$ and RV semi-amplitudes $(K_1, K_2)$. We can therefore immediately calculate the mass ratio of the two stars:

\begin{align}
\frac{M_2}{M_1} = \frac{K_1}{K_2}
\end{align}

We can also calculate the sum of the masses by considering Kepler's third law, namely:

\begin{align}
\frac{a^3}{P^2} &= \frac{G(M_1 + M_2)}{4\pi^2}
\end{align}

Both stars orbit their shared centre-of-mass with semi-major axes $a_1$ and $a_2$. However, with a frame of reference centred on the primary star, the secondary appears to orbit around it with semi-major axis $a=a_1 + a_2$ (or vice versa).

__Using Kepler's third law and the expressions for $K_1$ and $K_2$ above, show that the sum of the masses is given by the following:__

\begin{align}
(M_1 + M_2)\sin^3 i = \frac{P}{2\pi G}(1-e^2)^{3/2}(K_1 + K_2)^3
\end{align}

__and calculate the sum for THOR 42 assuming an inclination of $i=85^{\circ}$ and circular orbits. Use the  sum and ratio of the masses to calculate the individual masses in Solar units ($1 M_{\odot} = 1.989\times10^{30}\  \mathrm{kg}$).__

__Derive an expression for $a=a_1+a_2$ in terms of $(K_1,K_2,P,i,e)$, and use this to calculate the separation of the stars in Solar radii $(1 R_{\odot} = 6.957\times10^{8}\ \mathrm{m})$ and astronomical units $(1\ \mathrm{au} = 1.4959\times10^{11}\ \mathrm{m})$ assuming circular orbits and $i=85^{\circ}$.__

## E. Radii of THOR 42

<br>The sum and ratio of the relative radii (i.e. $R/a$, for semi-major axis $a$) can be extracted from the eclipse light curve. The most accurate values come from a full physical model of the system, taking into account things like limb darkening, non-spherical stars and star spots etc. Nevertheless, we can still extract reasonable values using nothing more than geometry and some basic assumptions.

The duration of the eclipses tells us about the sum of the radii, as the smaller, fainter star must transit the disk of the brighter star during primary eclipse, and is occulted by the primary star during secondary eclipse. The following discussion is taken from [work on transiting exoplanets](https://www.paulanthonywilson.com/exoplanets/exoplanet-detection-techniques/the-exoplanet-transit-method/) but is also applicable to eclipsing binaries. Consider the system as seen from Earth the moment the primary eclipse starts:

<div style="margin:50px">
<img src="transit.png" width="350px"/>
</div>

In our case $R_{\star} = R_1$ and $R_{p}= R_2$. The secondary star is moving right to left. Using Pythagoras, the length the secondary star (or planet) has to travel across the disk of the primary as projected onto the plane of sky can be expressed as

\begin{align}
2\ell = 2\sqrt{(R_1 + R_2)^2 - (b R_1)^2}
\end{align}

where $b=a\cos (i)/R_1$ is called the *impact parameter* of the eclipse and is 0 at the centre of the disk and 1 at the limb. For a completely edge-on orbit ($i=90^{\circ}$) the impact parameter is zero. Let us now consider how this looks in three dimensions:

<div style="margin:50px">
<img src="orbit.png" width="700px"/>
</div>

The secondary moves from $A$ to $B$ around its orbit, creating an angle $\alpha$ (measured in radians) with respect to the centre of the primary star. If we assume the orbit is circular (a very good assumption for THOR 42), the distance around an entire orbit is $2\pi a$, where $a$ is now the radius of the orbit. The arc length between $A$ and $B$ is $\alpha a$ and the distance along a straight line between $A$ and $B$ is $2\ell$, as calcuated above. 

From the triangle formed by $A$, $B$ and the centre of the star, 

\begin{align}
\sin(\frac{\alpha}{2}) = \frac{\ell}{a}
\end{align}

and therefore the transit duration $\Delta t$ (in time units) is given by:

\begin{align}
\Delta t & = P \frac{\alpha}{2\pi} \\
 & = \frac{P}{\pi}\sin^{-1}(\frac{\ell}{a}) \\
 & = \frac{P}{\pi}\sin^{-1}(\frac{1}{a}\sqrt{(R_1 + R_2)^2 - (b R_1)^2}) \\
 & = \frac{P}{\pi}\sin^{-1}(\frac{1}{a}\sqrt{(R_1 + R_2)^2 - (a \cos i)^2}) 
\end{align}

__From your phased light curve, measure the primary and secondary eclipse durations (do you expect them to be different?) and calculate an average value. Rearrange the above equation to show that the sum of the relative radii is given by__

\begin{align}
\frac{R_1}{a} + \frac{R_2}{a} & = \sqrt{\cos^2 i + \sin^2(\pi \Delta\phi)}
\end{align}

__for eclipse duration $\Delta\phi$ (now in phase units). Calculate the value for THOR 42 assuming an inclination of $i=85^{\circ}$. Using the value for $a$ you found in Part D, calculate the sum of the *absolute* radii, $R_1+R_2$.__

<br>

The last remaining parameter to determine is the ratio of the radii, $R_2/R_1$. We can obtain this from the eclipse depths, assuming that the eclipses are total. Since our eclipses are not flat-bottomed we know the system cannot be exactly at $i=90^{\circ}$. However, if $R_2$ is small enough it can still cause a near-total eclipse at $i<90^{\circ}$. 

Outside of the eclipses (and neglecting the effect of the spot) we see light from both stars, and the signal $B_0$  received by the telescope is

\begin{align}
B_0 & = c'\ (L_1 + L_2) \\
               & = c\ (R_1^2 F_1 + R_2^2 F_2)
\end{align}

where $(L_1,L_2)$ and $(F_1,F_2)$ are the luminosities (in W or erg s$^{-1}$) and surface fluxes (in W m$^{-2}$ or erg s$^{-1}$cm$^{-2}$) of the stars, respectively, and $c$ is a constant which depends on things like the distance to the system, the amount of interstellar absorption and atmopsheric extinction. During secondary eclipse when the secondary star is completely occulted by the primary, the signal is just that of the primary star:

\begin{align}
B_2 = c\ (R_1^2 F_1)
\end{align}

and during primary eclipse we add the two stars together then subtract the amount of light lost from the primary as the secondary passes across it:

\begin{align}
B_1 = c\ (R_1^2 F_1 + R_2^2 F_2 - R_2^2 F_1)
\end{align}

__Rearrange the above equations to show that the ratio of eclipse depths is equal to the surface flux ratio, namely__

\begin{align}
\frac{B_0-B_1}{B_0-B_2} = \frac{F_1}{F_2}
\end{align}

__and then use the fact that the luminosity $L=4\pi R^2 F$ to show that__

\begin{align}
\frac{R_2}{R_1} = \sqrt{\frac{L_2}{L_1}\frac{F_1}{F_2}}
\end{align}

__Measure the eclipse depths from the phased *TESS* light curve and calculate the ratio of the radii $R_2/R_1$.__ You can assume $B_0 = 1.0$ for both the primary and secondary eclipses (i.e. include the brightness decrease due to the spot in the total eclipse depth). This calculation also requires knowing the luminosity ratio of the two stars, $L_2/L_1$, of which we have some prior knowledge from spectroscopic observations. As seen at Earth, the secondary appears roughly 1/5 as bright as the primary at optical wavelengths, or more accurately, $L_2/L_1 = 0.22$.

__Combine the sum and ratio of the radii to derive the individual radii in Solar units $(1 R_{\odot} = 6.957\times10^{8}\ \mathrm{m})$.__

__Prepare a table in your report of your results. Include values for the period, inclination and semi-major axis of the system, as well as the individual masses and radii, with appropriate units and significant figures. Leave an extra column so you can compare your results to published values.__

<br>

## F. Age of THOR 42

Finally, we can use the masses and radii we have derived to estimate ages for each component. These should be similar to each other (co-evality) as we expect both stars to have been born together in the same stellar nursery.

Young, low-mass stars like those comprising THOR 42 are not yet in hydrostatic equilibrium, and are therefore slowly contracting down to their eventual main sequence radii. The regularity of this transition means we can use the measured radius to estimate an age for the star by comparing its progress to theoretical stellar [*isochrones*](https://ui.adsabs.harvard.edu/abs/2015A%26A...577A..42B/abstract), which join together models of constant age. Be aware that different suites of theoretical models make different physical assumptions and as a result can predict quite different ages for the same set of stars. 

__Add your mass and radius measurements to the figure below. Adjust the isochrone to estimate individual ages for both stars. Are they approximately co-eval? Is there a single age which is a reasonable match to both stars?__ 

THOR 42 is thought to be a member of the 32 Orionis Moving Group, which has an independent age estimate of $24\pm4\ \mathrm{Myr}$. __Are your ages consistent with this value and its uncertainty? Include a copy of the figure with the best-fitting isochrone in your report.__

<br>

In [ ]:
# Plot the interactive figure
MassRadiusDiagram()

# Part III: Conclusions

<br>

Input your mass, radius and separation measurements into the light curve simulator. You may assume circular orbits, an inclination of $i=85^{\circ}$ and temperatures of $T_1 = 3100\ \mathrm{K}$ and $T_2 = 3000\ \mathrm{K}$. 

__Neglecting the dip caused by the spot, does the simulated light curve broadly match the observed one? Does the system period predicted by the simulator agree with what you have derived from the light curve?__

__Save three plots showing the system as viewed from Earth at phases of $\phi=0$ (primary eclipse), $\phi=0.25$ (quadrature) and $\phi=0.5$ (secondary eclipse) to include in your report. Was the assertion of total eclipses valid at the assumed inclination of $i=85^{\circ}$?__

Finally, compare your measurements to those from a recent investigation of THOR 42 using many of the same observations ([Murphy et al. 2020, MNRAS, 491 4902](https://arxiv.org/pdf/1911.05925.pdf)). Table 6 in the article lists the relevant parameter estimates and their uncertainties. Table 9 lists age estimates from several sets of theoretical models; your age analysis above corresponds to the BHAC15 MRD ages.

__Add the published measurements to the table in your report and comment on any significant differences.__

__Briefly summarise your analysis and findings, answering the following questions:__

- What were the major sources of uncertainty in your analysis? This can include limitations of the data, as well as any assumptions made.
- How might you improve your parameter estimates using either (i) only the observations analysed here, or (ii) any new observations?

<br>

__Submit a PDF of your report, including the discussion in Part I,  via Moodle before the end of the session.__

<br><br><br>
